![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)

# Distributed computation
## ESIPE — INFO 3 — Option Logiciel

# Lab 2 : Web Server Log Analysis with Apache Spark

This lab will demonstrate how easy it is to perform web server log analysis with Apache Spark.

 
Server log analysis is an ideal use case for Spark.  It's a very large, common data source and contains a rich set of information.  Spark allows you to store your logs in files on disk cheaply, while still providing a quick and simple way to perform data analysis on them.  This homework will show you how to use Apache Spark on real-world text-based production logs and fully harness the power of that data.  Log data comes from many sources, such as web, file, and compute servers, application logs, user-generated content,  and can be used for monitoring servers, improving business and customer intelligence, building recommendation systems, fraud detection, and much more.


## How to complete this lab

 This assignment is broken up into sections with bite-sized examples for demonstrating Spark functionality for log processing. For each problem, you should start by thinking about the algorithm that you will use to *efficiently* process the log in a parallel, distributed manner. This means using the various [RDD](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD) operations along with [`lambda` functions](https://docs.scala-lang.org/overviews/scala-book/anonymous-functions.html#inner-main) that are applied at each worker.

 
This assignment consists of 4 parts:

- Part 1 : Apache Web Server Log file format
- Part 2 : Sample Analyses on the Web Server Log File with Spark Core
- Part 3 : Analyzing Web Server Log File with Spark SQL
- Part 4 : Exploring 404 Response Codes



##  Prerequisites : Spark Context configuration 

In [1]:
import org.apache.spark.sql._

val spark = SparkSession.builder
.appName("lab2_apache_log_text")
.master("local[*]")
.getOrCreate()
val sc = spark.sparkContext

spark = org.apache.spark.sql.SparkSession@7781eaad
sc = org.apache.spark.SparkContext@3e5b9544


org.apache.spark.SparkContext@3e5b9544

# Part 1 : Apache Web Server Log file format

If you're familiar with web servers at all, you'll recognize that this is in [Common Log Format](https://www.w3.org/Daemon/User/Config/Logging.html#common-logfile-format). The fields are:

_remotehost rfc931 authuser [date] "request" status bytes_

| field         | meaning                                                                |
| ------------- | ---------------------------------------------------------------------- |
| _remotehost_  | Remote hostname (or IP number if DNS hostname is not available).       |
| _rfc931_      | The remote logname of the user. We don't really care about this field. |
| _authuser_    | The username of the remote user, as authenticated by the HTTP server.  |
| _[date]_      | The date and time of the request.                                      |
| _"request"_   | The request, exactly as it came from the browser or client.            |
| _status_      | The HTTP status code the server sent back to the client.               |
| _bytes_       | The number of bytes (`Content-Length`) transferred to the client.      |



The log file entries produced in CLF will look something like this:

`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
Each part of this log entry is described below.

* `127.0.0.1`
This is the IP address (or host name, if available) of the client (remote host) which made the request to the server.

 
* `-`
The "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.

 
* `-`
The "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available.

 
* `[01/Aug/1995:00:00:01 -0400]`
The time that the server finished processing the request. The format is:

`[day/month/year:hour:minute:second timezone]`
  * day = 2 digits
  * month = 3 letters
  * year = 4 digits
  * hour = 2 digits
  * minute = 2 digits
  * second = 2 digits
  * zone = (\+ | \-) 4 digits
 
* `"GET /images/launch-logo.gif HTTP/1.0"`
This is the first line of the request string from the client. It consists of a three components: the request method (e.g., `GET`, `POST`, etc.), the endpoint (a [Uniform Resource Identifier](http://en.wikipedia.org/wiki/Uniform_resource_identifier)), and the client protocol version.

 
* `200`
This is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification ([RFC 2616](https://www.ietf.org/rfc/rfc2616.txt) section 10).

 
* `1839`
The last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).

 
Note that log files contain information supplied directly by the client, without escaping. Therefore, it is possible for malicious clients to insert control-characters in the log files, *so care must be taken in dealing with raw logs.*

 
## NASA-HTTP Web Server Log
For this assignment, we will use a data set from NASA Kennedy Space Center WWW server in Florida. The full data set is freely available (http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html) and contains two month's of all HTTP requests. We are using a subset that only contains several days worth of requests.



## 1. Parsing Each Log Line
Using the CLF as defined above, we create a regular expression pattern to extract the nine fields of the log. The function returns a pair consisting of a Row object and 1. If the log line fails to match the regular expression, the function returns a pair consisting of the log line string and 0. A '-' value in the content size field is cleaned up by substituting it with 0. The function converts the log line's date string into a `java.sql.Timestamp` object using the given `parse_apache_time` function.


We first define the case class `Log` which will be the structure of our log.

In [2]:
import java.sql.Timestamp
case class Log(
                host          : String,
                client_identd : String,
                user_id       : String,
                date_time     : Timestamp,
                method        : String,
                endpoint      : String,
                protocol      : String,
                response_code : Int,
                content_size  : Int
)

defined class Log


In [3]:
import scala.util.matching.Regex
//import datetime

//import org.apache.spark.sql.Row

val month_map = Map("Jan"-> 1, "Feb"-> 2, "Mar"->3, "Apr"->4, "May"->5, "Jun"->6, "Jul"->7,
  "Aug"->8,  "Sep"-> 9, "Oct"->10, "Nov"-> 11, "Dec"-> 12)

def parse_apache_time(s: String) = {
    /* Convert Apache time format into a Timestamp object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: Timestamp object (ignore timezone for now)
    */
    val year = s.substring(7,11).toInt
    val month = month_map(s.substring(3,6))
    val day = s.substring(0,2).toInt
    val hour = s.substring(12,14).toInt
    val min = s.substring(15,17).toInt
    val sec = s.substring(18,20).toInt
    Timestamp.valueOf(s"${year}-${month}-${day} ${hour}:${min}:${sec}")
}
                             
//A regular expression pattern to extract fields from the log line
val APACHE_ACCESS_LOG_PATTERN = new Regex("""^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)""")

val parseApacheLogLine = (logline : String) => {
    /* Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    */
    logline match {
        case APACHE_ACCESS_LOG_PATTERN(host, client_identd, user_id, date_time, method, endpoint, protocol, response_code, content_size) => {
            (Log(
                host,
                client_identd,
                user_id,
                parse_apache_time(date_time),
                method,
                endpoint,
                protocol,
                response_code.toInt,
                if (content_size == "-") 0 else content_size.toInt
            ),1)
        }
        case _ => (logline, 0)

    }
}

month_map = Map(Nov -> 11, Jul -> 7, Mar -> 3, Jan -> 1, Oct -> 10, Dec -> 12, Feb -> 2, May -> 5, Apr -> 4, Aug -> 8, Sep -> 9, Jun -> 6)
APACHE_ACCESS_LOG_PATTERN = ^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)
parseApacheLogLine = > (java.io.Serializable, Int) = <function1>


parse_apache_time: (s: String)java.sql.Timestamp


> (java.io.Serializable, Int) = <function1>

## 2. Configuration and Initial RDD Creation
We are ready to specify the input log file and create an RDD containing the parsed log file data. The log file has already been downloaded for you.

 
To create the primary RDD that we'll use in the rest of this assignment, we first load the text file using [`sc.textfile(logFile)`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.SparkContext@textFile(path:String,minPartitions:Int):org.apache.spark.rdd.RDD[String]) to convert each line of the file into an element in an RDD.

Next, we use [`map(parseApacheLogLine)`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD@map[U](f:T=%3EU)(implicitevidence$3:scala.reflect.ClassTag[U]):org.apache.spark.rdd.RDD[U]) to apply the parse function to each element (that is, a line from the log file) in the RDD and turn each line into a pair with the [`Log` case class].

Finally, we cache the RDD in memory since we'll use it throughout this notebook.



In [4]:
import org.apache.spark.rdd.RDD
val logFile = "../data/apache.access.log.PROJECT.lite"

def parseLogs() = {
    // Read and parse log file 
    val parsed_logs  = sc
                       .textFile(logFile)
                       .map(parseApacheLogLine)
                       .cache()

    val access_logs : RDD[Log] = parsed_logs
                       .filter(s => s._2 == 1)
                       .map(s => s._1.asInstanceOf[Log])
                       .cache()

    val failed_logs : RDD[String]  = (parsed_logs
                       .filter(s => s._2 == 0)
                       .map(s => s._1.asInstanceOf[String]))
    
    val failed_logs_count = failed_logs.count()
    
    if (failed_logs_count > 0) {
        println(s"Number of invalid logline: ${failed_logs.count()}")
        failed_logs.take(20).foreach(line => println(s"Invalid logline: ${line}"))
    }
        
    println(s"Read ${parsed_logs.count()} lines, successfully parsed ${access_logs.count()} lines, failed to parse ${failed_logs.count()} lines")
    (parsed_logs, access_logs, failed_logs)
}
    

val (parsed_logs, access_logs, failed_logs) = parseLogs()

Read 1000 lines, successfully parsed 1000 lines, failed to parse 0 lines


logFile = ../data/apache.access.log.PROJECT.lite
parsed_logs = MapPartitionsRDD[2] at map at <console>:43
access_logs = MapPartitionsRDD[4] at map at <console>:48
failed_logs = MapPartitionsRDD[6] at map at <console>:53


parseLogs: ()(org.apache.spark.rdd.RDD[(java.io.Serializable, Int)], org.apache.spark.rdd.RDD[Log], org.apache.spark.rdd.RDD[String])


MapPartitionsRDD[6] at map at <console>:53

# Part 2 : Sample Analyses on the Web Server Log File with Spark Core
 
Now that we have an RDD containing the log file as a set of Row objects, we can perform various analyses.

 
## 1. Content Size Statistics

 
Let's compute some statistics about the sizes of content being returned by the web server. In particular, we'd like to know what are the average, minimum, and maximum content sizes.

 
We can compute the statistics by applying a `map` to the `access_logs` RDD. The `lambda` function we want for the map is to extract the `content_size` field from the RDD. The map produces a new RDD containing only the `content_sizes` (one element for each Row object in the `access_logs` RDD). To compute the minimum and maximum statistics, we can use [`min()`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD@min()(implicitord:Ordering[T]):T) and [`max()`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD@max()(implicitord:Ordering[T]):T) functions on the new RDD. We can compute the average statistic by using the [`reduce`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD@reduce(f:(T,T)=%3ET):T) function with a `lambda` function that sums the two inputs, which represent two elements from the new RDD that are being reduced together. The result of the `reduce()` is the total content size from the log and it is to be divided by the number of requests as determined using the [`count()`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD@count():Long) function on the new RDD.



In [5]:
// TODO: Replace ??? with appropriate code
// Calculate statistics based on the content size
// HINT : RDD has been indexed in function parseApacheLogLine(). 
// You can now access content sizes using attribute .content_size
val content_sizes = access_logs.map(log => log.content_size).cache()
val content_sizes_mean = content_sizes.mean()
val content_sizes_min = content_sizes.min()
val content_sizes_max = content_sizes.max()
println(s"Content Size Avg: ${content_sizes_mean.round}, Min: ${content_sizes_min}, Max: ${content_sizes_max}")

Content Size Avg: 16234, Min: 0, Max: 887988


content_sizes = MapPartitionsRDD[7] at map at <console>:37
content_sizes_mean = 16233.652999999995
content_sizes_min = 0
content_sizes_max = 887988


887988

Creating the function for the tests

In [6]:
def assertEquals[A](expected : A, answer : A, error : String) = {
    if (expected equals answer) println("1 test passed")
    else error
}

assertEquals: [A](expected: A, answer: A, error: String)Any


In [7]:
// TEST Content size statistics
assertEquals((content_sizes_mean.round, content_sizes_min, content_sizes_max), (17532, 0, 3421948), "incorrect expected values")

incorrect expected values

## 2. Response Code Analysis

Next, lets make a count of the response codes that appear in the logs. As with the content size analysis, first we create a new RDD by using a `lambda` function to extract the `response_code` field from the `access_logs` RDD. The difference here is that we will use a [pair tuple](https://docs.scala-lang.org/tour/tuples.html) instead of just the field itself. Using a pair tuple consisting of the response code and 1 will let us count how many records have a particular response code. Using the new RDD, we perform a [`reduceByKey`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions@reduceByKey(func:(V,V)=%3EV):org.apache.spark.rdd.RDD[(K,V)]) function. `reduceByKey` performs a reduce on a per-key basis by applying the `lambda` function to each element, pairwise with the same key. We use the simple `lambda` function of adding the two values. Then, we cache the resulting RDD and create a list by using the [`take`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD@take(num:Int):Array[T]) function.

_Note_ : The expected method is similar of the word count approach developed in last week TP.

In [8]:
// TODO: Replace <FILL IN> with appropriate code
// Make a Response Code repartition count.
// HINT : you can access to the log response_code using attribute ".response_code".

val responseCodeToCount = access_logs.map(l => l.response_code -> 1)
                            .reduceByKey(_ + _)
val responseCodeToCountList = responseCodeToCount.collect()
println(s"Found ${responseCodeToCountList.length} response codes")
println(s"Response Code Counts: ${responseCodeToCountList.mkString}")

Found 4 response codes
Response Code Counts: (404,2)(200,923)(302,16)(304,59)


responseCodeToCount = ShuffledRDD[11] at reduceByKey at <console>:38
responseCodeToCountList = Array((404,2), (200,923), (302,16), (304,59))


Array((404,2), (200,923), (302,16), (304,59))

In [ ]:
// TEST : Response Code Analysis
assertEquals(responseCodeToCountList.toList.sorted, List((200, 940847), (302, 16244), (304, 79824), (403, 58), (404, 6185), (500, 2), (501, 17)), "Incorrect response code analysis")

## 3. Response Code Graphing with `Vegas`
Now, lets visualize the results from the last example. We can visualize the results from the last example using [`vegas`](https://www.vegas-viz.org/). First we need to extract the labels and fractions for the graph. We do this with two separate `map` functions with a `lambda` functions. The first `map` function extracts a list of of the response code values, and the second `map` function extracts a list of the per response code counts  divided by the total size of the access logs. Next, we creat the plot.

In [9]:
// TODO: Replace ??? with appropriate code
// Assign the response code labels using .map() and .collect() to "labels" variable.
// Assign the number of rows to variable "rows_count"
// Assign the proportional count to variable "fracs_count" (don't forget to collect).
val labels = responseCodeToCount.map(_._1).collect
val rows_count = access_logs.count
val fracs_count = responseCodeToCount.map(_._2).collect
println(labels.mkString(" "))
println(fracs_count.mkString(","))

404 200 302 304
2,923,16,59


labels = Array(404, 200, 302, 304)
rows_count = 1000
fracs_count = Array(2, 923, 16, 59)


Array(2, 923, 16, 59)

In [ ]:
// TEST : Response code graphing
assertEquals(fracs_count.toList.sorted, List(1.9172201841106543e-06, 1.629637156494056e-05, 5.5599385339208974e-05, 0.005929003419362198, 0.015571662335346735, 0.07652009198822443, 0.9019054292799784), "Incorrect proportions")

The following cells are just for importing Vegas

In [10]:
%AddDeps org.vegas-viz vegas_2.11 0.3.10 --transitive
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.10 --transitive

Marking org.vegas-viz:vegas_2.11:0.3.10 for download
Obtained 42 files
Marking org.vegas-viz:vegas-spark_2.11:0.3.10 for download
Obtained 44 files


In [11]:
implicit val render = vegas.render.ShowHTML(kernel.display.content("text/html", _))

render = <function1>


<function1>

In [12]:
import vegas._
import vegas.data.External._
import vegas.sparkExt._

Vegas("Response Code").
  mark(Bar).
  withData(labels.zip(fracs_count).map(x => Map("Response Code"->x._1,"fracs_count"->x._2))).
  encodeX("Response Code", Nom).
  encodeY("fracs_count", Quant).
  encodeColor(field="Response Code", dataType=Nominal).
  show

<iframe id="frame-vegas-3e6398e1-17f9-4c6b-a0b1-60f4164db929" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-3e6398e1-17f9-4c6b-a0b1-60f4164db929'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Response Code",
 "type" : "nominal"
 },
 "y" : {
 "field" : "fracs_count",
 "type" : "quantitative"
 },
 "color" : {
 "field" : "Response Code",
 "type" : "nominal"
 }
 },
 "description" : "Response Code",
 "data" : {
 "values" : [
 {
 "Response Code" : 404,
 "fracs_count" : 2
 },
 {
 "Response Code" : 200,
 "fracs_count" : 923
 },
 {
 "Response Code" : 302,
 "fracs_count" : 16
 },
 {
 "Response Code" : 304,
 "fracs_count" : 59
 }
 ]
 }
}
 }
 vg.embed("#vegas-3e6398e1-17f9-4c6b-a0b1-60f4164db929", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

## 4. Visualizing Endpoints

Now, lets visualize the number of hits to endpoints (URIs) in the log. To do that, perform a endpoints counts similar to response codes approach.

_Hint_ : To perform this task, we first create a new RDD by using a `lambda` function to extract the `endpoint` field from the `access_logs` RDD using a pair tuple consisting of the endpoint and 1 which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a `lambda` function that adds the two values. We then cache the results.

 
Next we visualize the results using `Vegas`. We previously imported the `Vegas` libraries, so we do not need to import it again. We perform two separate `map` functions with `lambda` functions. The first `map` function extracts a list of endpoint values, and the second `map` function extracts a list of the visits per endpoint values. Next, we create the line plot.



In [45]:
// TODO: Replace ??? with appropriate code
// Assign the endpoints <key, value> counts to variable "endpoints" using .map() and .reduceByKey().
// Assign the count to variables "end_counts" (don't forget to collect).
// Normally we would show the labels in the x axis but it doesn't work in the notebook
// However we only wanted to see the number of hits so the labels are not very important here

val endpoints = access_logs.map(_.endpoint)
val ends_counts = endpoints.map(_ -> 1).reduceByKey(_ + _).collect

Vegas("404 Response d by Day",width=800).
  mark(Bar).
  withData(ends_counts.zipWithIndex.map(x => Map("Endpoints"->x._1,"Number of Hits"->x._2))).
  encodeX("Endpoints", Nom).
  encodeY("Number of Hits", Quant).
  show


<iframe id="frame-vegas-44a65494-f97b-41a1-99cf-cfacfa8b67b2" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-44a65494-f97b-41a1-99cf-cfacfa8b67b2'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 800.0,
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Endpoints",
 "type" : "nominal"
 },
 "y" : {
 "field" : "Number of Hits",
 "type" : "quantitative"
 }
 },
 "description" : "404 Response d by Day",
 "data" : {
 "values" : [
 {
 "Endpoints" : "(/cgi-bin/imagemap/countdown70?183,288,1)",
 "Number of Hits" : 0
 },
 {
 "Endpoints" : "(/history/apollo/apollo-goals.txt,1)",
 "Number of Hits" : 1
 },
 {
 "Endpoints" : "(/images/shuttle-patch-logo.gif,4)",
 "Number of Hits" : 2
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-70/movies/movies.html,4)",
 "Number of Hits" : 3
 },
 {
 "Endpoints" : "(/history/gemini/gemini.html,3)",
 "Number of Hits" : 4
 },
 {
 "Endpoints" : "(/biomed/wwwicons/blue.gif,1)",
 "Number of Hits" : 5
 },
 {
 "Endpoints" : "(/persons/nasa-cm/mike-sm.gif,1)",
 "Number of Hits" : 6
 },
 {
 "Endpoints" : "(/icons/text.xbm,4)",
 "Number of Hits" : 7
 },
 {
 "Endpoints" : "(/shuttle/missions/51-c/mission-51-c.html,1)",
 "Number of Hits" : 8
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-68/news/sts-68-mcc-18.txt,1)",
 "Number of Hits" : 9
 },
 {
 "Endpoints" : "(/shuttle/missions/61-c/mission-61-c.html,1)",
 "Number of Hits" : 10
 },
 {
 "Endpoints" : "(/htbin/cdt_main.pl,4)",
 "Number of Hits" : 11
 },
 {
 "Endpoints" : "(/shuttle/technology/images/sts_spec_6-small.gif,1)",
 "Number of Hits" : 12
 },
 {
 "Endpoints" : "(/software/winvn/bluemarb.gif,10)",
 "Number of Hits" : 13
 },
 {
 "Endpoints" : "(/images/launchpalms-small.gif,1)",
 "Number of Hits" : 14
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-56/mission-sts-56.html,1)",
 "Number of Hits" : 15
 },
 {
 "Endpoints" : "(/shuttle/technology/sts-newsref/srb.html,1)",
 "Number of Hits" : 16
 },
 {
 "Endpoints" : "(/shuttle/resources/orbiters/columbia-logo.gif,1)",
 "Number of Hits" : 17
 },
 {
 "Endpoints" : "(/images/KSC-logosmall.gif,49)",
 "Number of Hits" : 18
 },
 {
 "Endpoints" : "(/facilities/lc39a.html,4)",
 "Number of Hits" : 19
 },
 {
 "Endpoints" : "(/software/winvn/wvlarge.gif,2)",
 "Number of Hits" : 20
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-75/mission-sts-75.html,1)",
 "Number of Hits" : 21
 },
 {
 "Endpoints" : "(/shuttle/technology/images/launch_sites_8-small.gif,1)",
 "Number of Hits" : 22
 },
 {
 "Endpoints" : "(/cgi-bin/imagemap/countdown70?61,223,1)",
 "Number of Hits" : 23
 },
 {
 "Endpoints" : "(/history/apollo/apollo-16/apollo-16-patch-small.gif,1)",
 "Number of Hits" : 24
 },
 {
 "Endpoints" : "(/cgi-bin/imagemap/countdown70?67,155,1)",
 "Number of Hits" : 25
 },
 {
 "Endpoints" : "(/history/apollo/apollo-1/images/66HC1519.gif,2)",
 "Number of Hits" : 26
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-64/mission-sts-64.html,3)",
 "Number of Hits" : 27
 },
 {
 "Endpoints" : "(/history/apollo/apollo-1/images/apollo-1-crew.gif,1)",
 "Number of Hits" : 28
 },
 {
 "Endpoints" : "(/icons/image.xbm,7)",
 "Number of Hits" : 29
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-73/mission-sts-73.html,1)",
 "Number of Hits" : 30
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-71/movies/sts-71-launch-3.mpg,2)",
 "Number of Hits" : 31
 },
 {
 "Endpoints" : "(/shuttle/missions/51-d/mission-51-d.html,1)",
 "Number of Hits" : 32
 },
 {
 "Endpoints" : "(/shuttle/missions/sts-68/news/sts-68-mcc-10.txt,1)",
 "

endpoints = MapPartitionsRDD[137] at map at <console>:88
ends_counts = Array((/cgi-bin/imagemap/countdown70?183,288,1), (/history/apollo/apollo-goals.txt,1), (/images/shuttle-patch-logo.gif,4), (/shuttle/missions/sts-70/movies/movies.html,4), (/history/gemini/gemini.html,3), (/biomed/wwwicons/blue.gif,1), (/persons/nasa-cm/mike-sm.gif,1), (/icons/text.xbm,4), (/shuttle/missions/51-c/mission-51-c.html,1), (/shuttle/missions/sts-68/news/sts-68-mcc-18.txt,1), (/shuttle/missions/61-c/mission-61-c.html,1), (/htbin/cdt_main.pl,4), (/shuttle/technology/images/sts_spec_6-small.gif,1), (/software/winvn/bluemarb.gif,10), (/images/launchpalms-small.gif,1), (/shuttle/missions/sts-56/mission-sts-56.html,1), (/shuttle/technology/sts-newsref/srb....


Array((/cgi-bin/imagemap/countdown70?183,288,1), (/history/apollo/apollo-goals.txt,1), (/images/shuttle-patch-logo.gif,4), (/shuttle/missions/sts-70/movies/movies.html,4), (/history/gemini/gemini.html,3), (/biomed/wwwicons/blue.gif,1), (/persons/nasa-cm/mike-sm.gif,1), (/icons/text.xbm,4), (/shuttle/missions/51-c/mission-51-c.html,1), (/shuttle/missions/sts-68/news/sts-68-mcc-18.txt,1), (/shuttle/missions/61-c/mission-61-c.html,1), (/htbin/cdt_main.pl,4), (/shuttle/technology/images/sts_spec_6-small.gif,1), (/software/winvn/bluemarb.gif,10), (/images/launchpalms-small.gif,1), (/shuttle/missions/sts-56/mission-sts-56.html,1), (/shuttle/technology/sts-newsref/srb....

## 5. Top 10 transferred bytes hosts

Now, let's answer the following question. Who are the top 10 hosts in terms of transferred bytes (content size) ? 

To perform this task, use RDD transformations `map`, `reduceByKey` and [`takeOrdered`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD@takeOrdered(num:Int)(implicitord:Ordering[T]):Array[T]).

In [24]:
// TODO: Replace ??? with appropriate code
// Assign the top 10 hosts considering transfered bytes to the variable "top_10_hosts".
val top_10_hosts = access_logs
  .map(l => l.host -> l.content_size)
  .reduceByKey(_ + _)
  .takeOrdered(10)(Ordering.by[(String, Int), Int](-_._2))

top_10_hosts = Array((uplherc.upl.com,2417420), (haraway.ucet.ufl.edu,2146536), (www-d3.proxy.aol.com,905926), (ad11-061.compuserve.com,793953), (pm9.j51.com,760823), (piweba3y.prodigy.com,557026), (rpgopher.aist.go.jp,413561), (133.43.96.45,410971), (dialip129.gov.bc.ca,379590), (www-d1.proxy.aol.com,358850))


Array((uplherc.upl.com,2417420), (haraway.ucet.ufl.edu,2146536), (www-d3.proxy.aol.com,905926), (ad11-061.compuserve.com,793953), (pm9.j51.com,760823), (piweba3y.prodigy.com,557026), (rpgopher.aist.go.jp,413561), (133.43.96.45,410971), (dialip129.gov.bc.ca,379590), (www-d1.proxy.aol.com,358850))

In [ ]:
// TEST : Top 10 tranfered bytes hosts
assertEquals(top_10_hosts.length, 10, "top_10_hosts had to be length 10")
assertEquals(top_10_hosts.toList, List("news.ti.com", "www-relay.pa-x.dec.com", "piweba5y.prodigy.com", "e659229.boeing.com", "piweba3y.prodigy.com", "www-c2.proxy.aol.com", "163.206.89.4", "www-b3.proxy.aol.com", "webgate1.mot.com", "gatekeeper.cca.rockwell.com"), "Incorrect top 10 hosts")

# Part 3 : Analyzing Web Server Log File with Spark SQL
 
Now it is time to perform advanced analytics on web server log files using Spark SQL.



## 1. Transform Spark RDD to Spark SQL dataframe

In order to use Spark SQL functionalities, you need to transform your logs data spark RDD to a Spark SQL dataframe. To perform this task, you can refer to the Spark lectures provided in classroom or spark documentation. After dataframe load, make sure that dataframe column casting is correct using method `printSchema`.

In [28]:
// TODO: Replace <FILL IN> with appropriate code
// Transform the access_logs spark RDD into Spark SQL Dataframe.
// Hint : Use the toDF method.
import org.apache.spark._

val spark = SparkSession.builder
.appName("lab1_word_count_text")
.master("local[*]")
.getOrCreate()
val sc = spark.sparkContext

import spark.implicits._

val logs_df: Dataset[Log] = access_logs.toDS
logs_df.cache()
logs_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- client_identd: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- date_time: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- response_code: integer (nullable = false)
 |-- content_size: integer (nullable = false)



spark = org.apache.spark.sql.SparkSession@7781eaad
sc = org.apache.spark.SparkContext@3e5b9544
logs_df = [host: string, client_identd: string ... 7 more fields]


<console>:6: error: Symbol 'type scala.AnyRef' is missing from the classpath.
This symbol is required by 'class org.apache.spark.sql.catalyst.QualifiedTableName'.
Make sure that type AnyRef is in your classpath and check for conflicting dependencies with `-Ylog-classpath`.
A full rebuild may help if 'QualifiedTableName.class' was compiled against an incompatible version of scala.
  lazy val $print: String =  {
           ^


[host: string, client_identd: string ... 7 more fields]

In [29]:
// TEST : Transform Spark RDD to Spark SQL dataframe
assertEquals(logs_df.getClass.toString, "class org.apache.spark.sql.Dataset", "logs_df is not a Spark Dataset.")
assertEquals(logs_df.dtypes.toList.sortBy(_._1) , List(("client_identd", "StringType"), ("content_size", "IntegerType"), ("date_time", "TimestampType"), ("endpoint", "StringType"), ("host", "StringType"), ("method", "StringType"), ("protocol", "StringType"), ("response_code", "IntegerType"), ("user_id", "StringType")), "Dataframe casting is not correct.")

1 test passed
1 test passed


()

## 2. Top 10 error endpoints

What are the top twenty paths which did not have return code 200? Create a sorted list containing the paths and the number of times that they were accessed with a non-200 return code and show the top ten.

Think about the steps that you need to perform to determine which paths did not have a 200 return code, how you will uniquely count those paths and sort the list.

In [46]:
// TODO: Replace ??? with appropriate code
// Assign logs with status different than 200 to variable "not200_df".
// Assign the counts per endpoint dataframe to variable "logs_sums_df". 
// Descending sort the logs_sum_df and assign the result to variable "sorted_logs_sum_df".
// Collect the top 10 endpoints and assign it to variable "top_ten_err_urls".
// Hint : You will need to use methods .groupBy() and .sort() to achieve this task.
// Note : You are welcome to structure your solution in a different way, so long as
// you ensure the variables used in the next Test section are defined (ie. logs_sum_df, top_ten_err_urls).

import org.apache.spark.sql.functions._

val not200_df = logs_df.where($"response_code" !== lit(200))
val logs_sum_df = not200_df.groupBy($"endpoint").count
val sorted_logs_sum_df = logs_sum_df.orderBy($"count".desc)
val top_ten_err_urls = sorted_logs_sum_df.take(10).toList

// Display
println("Top Ten failed URLs:")
sorted_logs_sum_df.show(10, false)
println(top_ten_err_urls.mkString(" "))

Top Ten failed URLs:
+---------------------------------------+-----+
|endpoint                               |count|
+---------------------------------------+-----+
|/images/NASA-logosmall.gif             |9    |
|/images/KSC-logosmall.gif              |6    |
|/shuttle/countdown/count70.gif         |4    |
|/images/MOSAIC-logosmall.gif           |4    |
|/images/USA-logosmall.gif              |4    |
|/images/WORLD-logosmall.gif            |4    |
|/images/ksclogo-medium.gif             |2    |
|/shuttle/countdown/                    |2    |
|/history/apollo/images/apollo-logo1.gif|2    |
|/icons/blank.xbm                       |2    |
+---------------------------------------+-----+
only showing top 10 rows

[/images/NASA-logosmall.gif,9] [/images/KSC-logosmall.gif,6] [/shuttle/countdown/count70.gif,4] [/images/MOSAIC-logosmall.gif,4] [/images/USA-logosmall.gif,4] [/images/WORLD-logosmall.gif,4] [/images/ksclogo-medium.gif,2] [/shuttle/countdown/,2] [/images/launch-logo.gif,2] [/icons

not200_df = [host: string, client_identd: string ... 7 more fields]
logs_sum_df = [endpoint: string, count: bigint]
sorted_logs_sum_df = [endpoint: string, count: bigint]
top_ten_err_urls = List([/images/NASA-logosmall.gif,9], [/images/KSC-logosmall.gif,6], [/shuttle/countdown/count70.gif,4], [/images/MOSAIC-logosmall.gif,4], [/images/USA-logosmall.gif,4], [/images/WORLD-logosmall.gif,4], [/images/ksclogo-medium.gif,2], [/shuttle/countdown/,2], [/images/launch-logo.gif,2], [/icons/blank.xbm,2])


List([/images/NASA-logosmall.gif,9], [/images/KSC-logosmall.gif,6], [/shuttle/countdown/count70.gif,4], [/images/MOSAIC-logosmall.gif,4], [/images/USA-logosmall.gif,4], [/images/WORLD-logosmall.gif,4], [/images/ksclogo-medium.gif,2], [/shuttle/countdown/,2], [/images/launch-logo.gif,2], [/icons/blank.xbm,2])

In [ ]:
// TEST Top ten error endpoints
import org.apache.spark.sql._
assertEquals(logs_sum_df.count(), 7689, "incorrect count for endpointSum")
assertEquals(top_ten_err_urls.toList, List(Row(s"/images/NASA-logosmall.gif", 8761), Row(s"/images/KSC-logosmall.gif", 7236), Row(s"/images/MOSAIC-logosmall.gif", 5197), Row(s"/images/USA-logosmall.gif", 5157), Row(s"/images/WORLD-logosmall.gif", 5020), Row(s"/images/ksclogo-medium.gif", 4728), Row(s"/history/apollo/images/apollo-logo1.gif", 2907), Row(s"/images/launch-logo.gif", 2811), Row(s"/", 2199), Row(s"/images/ksclogosmall.gif", 1622)), "incorrect Top Ten failed URLs (topTenErrURLs)")

## 3. Number of Unique Hosts

How many unique hosts are there in the entire log?

 
Think about the steps that you need to perform to count the number of different hosts in the log.



In [33]:
// TODO: Replace ??? with appropriate code
// Note : There are several ways to achieve this task.
val unique_host_count = logs_df.select($"host").distinct.count
println(s"Unique hosts: ${unique_host_count}")

Unique hosts: 102


unique_host_count = 102


102

In [ ]:
// TEST Number of unique hosts
assertEquals(unique_host_count, 54507, "incorrect unique_host_count")

## 4. Extract the date day using an UDF

In the next questions, we will compute the number of unique Daily hosts given the day of the month. To perform this following task, we need to create a new column which contains the day of the month (from 01 to 31). Fortunately, the day of the month is contained in the column `date_time` pattern.

_Note_ : There are several ways to achieve this task including using method `dayofmonth` from spark.sql.functions. In this exercise, we will use a UDF based solution in order to make you practice with the UDF concept. It is strongly recommended for you to look at the UDF slides provided during lecture 3.

_Note_ : Since the log only covers a single month, you can ignore the month.

In [39]:
// TODO: Replace <FILL IN> with appropriate code
// Complete the function 'date_to_day' in order to return the day of the month.
// Wrap the function into and UDF object. Assign the result to variable "my_udf".
// Apply the UDF on logs_df using method .WithColumn(). The resulting column has to be named "day". Assign the resulting
// dataframe to variable "logs_df_with_day".
import org.apache.spark.sql.functions.udf

val date_to_day = (date : Timestamp) => {
    /*
    Extracts the day of month in variable date.
    date : timestamp with pattern "yyyy-mm-dd hh:mm:ss"
    returns : day of the month (pattern 'dd')
    */
    val str_date = date.toString
    str_date.substring(8, 10)
}
    

// testing the function
assertEquals(date_to_day(Timestamp.valueOf("1995-08-01 00:00:07")), "01", "function date_to_day is not correct.")
assertEquals(date_to_day(Timestamp.valueOf("2017-09-17 10:00:07")), "17", "function date_to_day is not correct.")

// Wrap in a UDF and apply on logs_df
val date_to_day_udf = udf[String, Timestamp](date_to_day)
val logs_df_with_day = logs_df.withColumn("day", date_to_day_udf($"date_time"))
// logs_df_with_day.show(5)

1 test passed
1 test passed


date_to_day = > String = <function1>
my_udf = UserDefinedFunction(<function1>,StringType,Some(List(TimestampType)))
logs_df_with_day = [host: string, client_identd: string ... 8 more fields]


[host: string, client_identd: string ... 8 more fields]

In [38]:
%%dataframe
logs_df_with_day

host,client_identd,user_id,date_time,method,endpoint,protocol,response_code,content_size,day
in24.inetnebr.com,-,-,1995-08-01 00:00:01.0,GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,HTTP/1.0,200,1839,01
uplherc.upl.com,-,-,1995-08-01 00:00:07.0,GET,/,HTTP/1.0,304,0,01
uplherc.upl.com,-,-,1995-08-01 00:00:08.0,GET,/images/ksclogo-medium.gif,HTTP/1.0,304,0,01
uplherc.upl.com,-,-,1995-08-01 00:00:08.0,GET,/images/MOSAIC-logosmall.gif,HTTP/1.0,304,0,01
uplherc.upl.com,-,-,1995-08-01 00:00:08.0,GET,/images/USA-logosmall.gif,HTTP/1.0,304,0,01
ix-esc-ca2-07.ix.netcom.com,-,-,1995-08-01 00:00:09.0,GET,/images/launch-logo.gif,HTTP/1.0,200,1713,01
uplherc.upl.com,-,-,1995-08-01 00:00:10.0,GET,/images/WORLD-logosmall.gif,HTTP/1.0,304,0,01
slppp6.intermind.net,-,-,1995-08-01 00:00:10.0,GET,/history/skylab/skylab.html,HTTP/1.0,200,1687,01
piweba4y.prodigy.com,-,-,1995-08-01 00:00:10.0,GET,/images/launchmedium.gif,HTTP/1.0,200,11853,01
slppp6.intermind.net,-,-,1995-08-01 00:00:11.0,GET,/history/skylab/skylab-small.gif,HTTP/1.0,200,9202,01


In [47]:
// TEST Extract the month day using an UDF
val distinct_days = logs_df_with_day.select("day").distinct().collect()
distinct_days.length
//assertEquals(distinct_days.length, 21, "it seems that UDF is misapplied")

distinct_days = Array([01])


1

## 5. Number of Unique Daily Hosts

For an advanced exercise, let's determine the number of unique hosts in the entire log on a day-by-day basis. This computation will give us counts of the number of unique daily hosts. We'd like a DataFrame sorted by increasing day of the month which includes the day of the month and the associated number of unique hosts for that day. Make sure you cache the resulting DataFrame `daily_hosts_df` so that we can reuse it in the next exercise.

Think about the steps that you need to perform to count the number of different hosts that make requests *each* day.
*Since the log only covers a single month, you can ignore the month.*  You may want to use the `day` column you have computed in the previous task.

In [42]:
// TODO: Replace ??? with appropriate code
// Select columns "day" and "host" from "logs_df_with_day" dataframe and assign the result to "day_to_host_pair_df".
// Remove duplicates <"day", "host"> duplicates using method .distinct(). Assign the result to day_group_hosts_df.
// Group by day and count the distinct hosts. Assign the result dataframe to "daily_hosts_df".
// Sort per day and assign the result to "daily_hosts_df_sorted".
// Collect the result to driver. Assign the result to variable "daily_hosts_list".

val day_to_host_pair_df = logs_df_with_day.select("day", "host")
val day_group_hosts_df = day_to_host_pair_df.distinct
val daily_hosts_df = day_group_hosts_df.groupBy("day").count
val daily_hosts_df_sorted = daily_hosts_df.orderBy($"day" desc)
val daily_hosts_list = daily_hosts_df_sorted.collect

println("Unique hosts per day:")
daily_hosts_df_sorted.show(30, false)
daily_hosts_df.cache()

Unique hosts per day:
+---+-----+
|day|count|
+---+-----+
|01 |102  |
+---+-----+



day_to_host_pair_df = [day: string, host: string]
day_group_hosts_df = [day: string, host: string]
daily_hosts_df = [day: string, count: bigint]
daily_hosts_df_sorted = [day: string, count: bigint]
daily_hosts_list = Array([01,102])


[day: string, count: bigint]

In [ ]:
// TEST Number of unique daily hosts (3c)
assertEquals(daily_hosts_df.count(), 21, "incorrect dailyHosts.count()")
assertEquals(daily_hosts_list.toList, List(Row("01", 2582), Row("03", 3222), Row("04", 4190), Row("05", 2502), Row("06", 2537), Row("07", 4106), Row("08", 4406), Row("09", 4317), Row("10", 4523), Row("11", 4346), Row("12", 2864), Row("13", 2650), Row("14", 4454), Row("15", 4214), Row("16", 4340), Row("17", 4385), Row("18", 4168), Row("19", 2550), Row("20", 2560), Row("21", 4134), Row("22", 4456)), "incorrect dailyHostsList")
assertEquals(daily_hosts_df.storageLevel.useMemory, true, "incorrect dailyHosts.is_cached")

In [48]:
// plotting your results :

Vegas("404 Response Codes by Day").
  mark(Line).
  withDataFrame(daily_hosts_df_sorted).
  encodeX("day", Nom, axis=Axis(title="Day")).
  encodeY("count", Quant, axis=Axis(title="Hosts")).
  show


<iframe id="frame-vegas-669be0a9-f7d0-4773-9c0a-aece69430028" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-669be0a9-f7d0-4773-9c0a-aece69430028'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "axis" : {
 "title" : "Day"
 },
 "field" : "day",
 "type" : "nominal"
 },
 "y" : {
 "axis" : {
 "title" : "Hosts"
 },
 "field" : "count",
 "type" : "quantitative"
 }
 },
 "description" : "404 Response Codes by Day",
 "data" : {
 "values" : [
 {
 "day" : "01",
 "count" : 102
 }
 ]
 }
}
 }
 vg.embed("#vegas-669be0a9-f7d0-4773-9c0a-aece69430028", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

## 6. Mean Transfered Bytes per status code categories

For an advanced exercise, let's determine the mean transfered bytes per status code category. Remember that :
- Code beginning with a `2` means a request resulted in a successful response.
- Code beginning with a `3` means a request resulted in a redirection.
- Code beginning with a `4` means a request resulted in a client error.
- Code beginning with a `5` means a request resulted in a server error.

For every of these four category, compute the mean transfered bytes (content size).

_Hint_ : There are many ways to compute the code category column including the use of an UDF. You can choose the method you prefer. Feel free to refer to spark documentation and StackOverflow posts in order to find functions or informations you are searching for.

In [44]:
// TODO: Replace ??? with appropriate code
// Add a new colummn to logs_df contaning the code category. Assign the result to 'logs_df_with_code_category'.
// Group the logs per 'code category' computed in first subtask. Assign the result to 'logs_df_groupby_code_category'.
// Compute the content size mean per category and return the result to the driver. Assign the result 
// to 'content_size_per_code_category'.

import org.apache.spark.sql.function._

val logs_df_with_code_category = logs_df
    .withColumn("code_category", substring($"response_code".cast("string"), 0, 1) )
val logs_df_groupby_code_category = logs_df_with_code_category.groupBy("code_category")
val logs_df_agg_content_size = logs_df_groupby_code_category.agg(mean($"content_size"))
val content_size_per_code_category = logs_df_agg_content_size.collect.toList

logs_df_with_code_category = [host: string, client_identd: string ... 8 more fields]
logs_df_groupby_code_category = RelationalGroupedDataset: [grouping expressions: [code_category: string], value: [host: string, client_identd: string ... 8 more fields], type: GroupBy]
logs_df_agg_content_size = [code_category: string, avg(content_size): double]
content_size_per_code_category = List([3,20.586666666666666], [4,0.0], [2,17586.250270855904])


List([3,20.586666666666666], [4,0.0], [2,17586.250270855904])

In [ ]:
val round_result = content_size_per_code_category.map({case Row(x: Int,y: Double) => (x,(y*10).round/10.0)})
assertEquals(content_size_per_code_category.length, 4, "error : length has to be 4.")
assertEquals(round_result.toList, List((3, 14.4), (5, 10.4), (4, 0.0), (2, 19436.9)), "incorrect mean size content per category.")

# Part 4 : Exploring 404 Response Codes
 
Let's drill down and explore the error 404 response code records. 404 errors are returned when an endpoint is not found by the server (i.e., a missing page or object). During this part, you are free to complete the tasks using `Spark Core` and `access_logs` RDD or using `Spark SQL` and `logs-df` dataframe.

_Note_ : Do not forget to cache your RDD / Dataframe in memory in order to reduce computing time.

## 1. Counting 404 Response Codes
 
How many 404 records are there in the logs? Assign the result to variable `badRecords`.

In [ ]:
// TODO: Replace ??? with appropriate code

badRecords = ???
badRecords_count = ???

println(s"Found ${badRecordsSQL_count} 404 URLs")

In [ ]:
// TEST Counting 404 (4a)
assertEquals(badRecords_count, 6185, "incorrect badRecords_count.count()")

## 2. Listing The top-15 404 Response Code endpoints

Get the top 15 endpoints that return 404 errors. Assign the result to variable `top_15_404`.

_Note_ : variable `top_15_404` has to be a list only containing top 404 error endpoints.




In [ ]:
// TODO: Replace ??? with appropriate code

???

println(s"404 Top 15 URLS: ${top_15_404.mkString(" ")}")

In [ ]:
// TEST Listing 404 records (4.2)
assertEquals(top_15_404.length, 15, "top_15_404 length has to be 15.")
assertEquals(top_15_404.toList, List("/pub/winvn/readme.txt", "/pub/winvn/release.txt", "/shuttle/missions/STS-69/mission-STS-69.html", "/images/nasa-logo.gif", "/elv/DELTA/uncons.htm", "/shuttle/missions/sts-68/ksc-upclose.gif", "/history/apollo/sa-1/sa-1-patch-small.gif", "/images/crawlerway-logo.gif", "/://spacelink.msfc.nasa.gov", "/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif", "/history/apollo/a-001/a-001-patch-small.gif", "/images/Nasa-logo.gif", "/shuttle/resources/orbiters/atlantis.gif", "/history/apollo/images/little-joe.jpg", "/images/lf-logo.gif"), "top_15_404 not correct")


## 3. Listing the Top Twenty-five 404 Response Code Hosts

Instead of looking at the endpoints that generated 404 errors, let's look at the hosts that encountered 404 errors. Using the RDD / Dataframe containing only log records with a 404 response code that you cached in part (4.1), print out a list of the top twenty-five hosts that generate the most 404 errors. Assign the result to variable `errHostsTop25` and return your results to the driver. 



In [ ]:
// TODO: Replace ??? with appropriate code

???

println(s"Top 25 hosts that generated errors: ${errHostsTop25.mkString(" ")}")

In [ ]:
// TEST Top twenty-five 404 response code hosts (4.3)
assertEquals(errHostsTop25.length, 25, "length of errHostsTop25 is not 25")
assertEquals(errHostsTop25.toSet ,Set(("maz3.maz.net", 39), ("piweba3y.prodigy.com", 39), ("gate.barr.com", 38), ("m38-370-9.mit.edu", 37), ("ts8-1.westwood.ts.ucla.edu", 37), ("nexus.mlckew.edu.au", 37), ("204.62.245.32", 33), ("163.206.104.34", 27), ("spica.sci.isas.ac.jp", 27), ("www-d4.proxy.aol.com", 26), ("www-c4.proxy.aol.com", 25), ("203.13.168.24", 25), ("203.13.168.17", 25), ("internet-gw.watson.ibm.com", 24), ("scooter.pa-x.dec.com", 23), ("crl5.crl.com", 23), ("piweba5y.prodigy.com", 23), ("onramp2-9.onr.com", 22), ("slip145-189.ut.nl.ibm.net", 22), ("198.40.25.102.sap2.artic.edu", 21), ("gn2.getnet.com", 20), ("msp1-16.nas.mr.net", 20), ("isou24.vilspa.esa.es", 19), ("dial055.mbnet.mb.ca", 19), ("tigger.nashscene.com", 19)), "incorrect errHostsTop25")


## 4. Listing 404 Response Codes per Day

Let's explore the 404 records temporally. Break down the 404 requests by day (`cache()` the RDD `errDateSorted`) and get the daily counts sorted by day as a list. Assign the result to variable `errByDate` and return the result to driver.

*Since the log only covers a single month, you can ignore the month in your checks.*



In [ ]:
// TODO: Replace <FILL IN> with appropriate code

???

println(s"404 Errors by day: ${errByDate.mkString(" ")}")

In [ ]:
// TEST 404 response codes per day (4.4)
assertEquals(errByDate.toList, List((1, 243), (3, 303), (4, 346), (5, 234), (6, 372), (7, 532), (8, 381), (9, 279), (10, 314), (11, 263), (12, 195), (13, 216), (14, 287), (15, 326), (16, 258), (17, 269), (18, 255), (19, 207), (20, 312), (21, 305), (22, 288)), "incorrect errByDate")

## 5. Visualizing the 404 Response Codes by Day

Using the results from the previous exercise, use `Vegas` to plot a "Line" or "Bar" graph of the 404 response codes by day.

In [ ]:
// TODO: Replace ??? with appropriate code
val daysWithErrors404 = ???
val errors404ByDay = ???

In [ ]:
// TEST Visualizing the 404 Response Codes by Day (4f)
assertEquals(daysWithErrors404.toList, List(1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), "incorrect daysWithErrors404")
assertEquals(errors404ByDay.toList, List(243, 303, 346, 234, 372, 532, 381, 279, 314, 263, 195, 216, 287, 326, 258, 269, 255, 207, 312, 305, 288), "incorrect errors404ByDay")

In [ ]:
Vegas("404 Response Codes by Day").
  mark(Line).
  withData(daysWithErrors404.zip(errors404ByDay).map(x => Map("Day" -> x._1, "404 errors" -> x._2 ))).
  encodeX("Day", Nom).
  encodeY("404 errors", Quant).
  show


## 6. Hourly 404 Response Codes

Using the RDD / Dataframe `badRecords` you cached in the part (4.1) and by hour of the day and in increasing order, create an RDD / Dataframe containing how many requests had a 404 return code for each hour of the day (midnight starts at 0). Cache the resulting RDD hourRecordsSorted. Assign the result to variable `errHourList`and don't forget to return the result to the driver.

_Hint_ : Concerning the Spark SQL approach, multiple methods exist including the use of an UDF similar to exercise (3.4).

In [ ]:
// TODO: Replace <FILL IN> with appropriate code

???

println(s"Top hours for 404 requests: ${errHourList.mkString(" ")}")

In [ ]:
// TEST Hourly 404 response codes (4h)
assertEquals(errHourList.toList, List((0, 175), (1, 171), (2, 422), (3, 272), (4, 102), (5, 95), (6, 93), (7, 122), (8, 199), (9, 185), (10, 329), (11, 263), (12, 438), (13, 397), (14, 318), (15, 347), (16, 373), (17, 330), (18, 268), (19, 269), (20, 270), (21, 241), (22, 234), (23, 272)), "incorrect errHourListCore")

## 7. Visualizing the 404 Response Codes by Hour

Using the results from the previous exercise, use `Vegas` to plot a "Line" or "Bar" graph of the 404 response codes by hour.



In [ ]:
// TODO: Replace <FILL IN> with appropriate code

val hoursWithErrors404 = ???
val errors404ByHours = ???

In [ ]:
// TEST Visualizing the 404 Response Codes by Hour (4i)
assertEquals(hoursWithErrors404.toList, List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23), "incorrect hoursWithErrors404")
assertEquals(errors404ByHours.toList, List(175, 171, 422, 272, 102, 95, 93, 122, 199, 185, 329, 263, 438, 397, 318, 347, 373, 330, 268, 269, 270, 241, 234, 272), "incorrect errors404ByHours")

In [ ]:
Vegas("404 Response Codes by Hour").
  mark(Line).
  withData(hoursWithErrors404.zip(errors404ByHours).map(x => Map("hour" -> x._1, "404 errors" -> x._2 ))).
  encodeX("hour", Nom).
  encodeY("404 errors", Quant).
  encodeDetailFields(Field(field="symbol", dataType=Nominal)).
  show
